# Custom Kernel Tutorial
> Revision
> - Created on 21/01/2025, Cheng Zhang: PyTorch 2.5.0, CUDA 12.3
> - Fixed and Tested on 05/02/2025, Cheng Zhang: PyTorch 2.6.0, CUDA 12.5

## Env Setup in Colab

Check if Colab is connected to a NVIDIA Tesla T4 or Ada L4 GPU (L4 is faster), if not, change Colab runtime to T4 or L4.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('❌ Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb 12 17:37:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Install dependencies

In [2]:
! pip install tox ninja torch numpy scipy rust-just pytest transformers
! pip install -U build

# git_token = "Your Git Token"
! git clone --recurse-submodules https://github.com/DeepWok/mase-cuda.git
%cd mase-cuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Build & Run C++ Test

In [3]:
!just build-cu-test

# python
if [ -d /content/mase-cuda/dist ]; then rm -r /content/mase-cuda/dist; fi
if [ -d /content/mase-cuda/src/mase_cuda.egg-info ]; then rm -r /content/mase-cuda/src/mase_cuda.egg-info; fi
# all
if [ -d /content/mase-cuda/build ]; then rm -r /content/mase-cuda/build; fi
echo $(which cmake)
/usr/local/bin/cmake
cmake -D BUILD_TESTING=ON -D CUDA_ARCHITECTURES=native -B build -S .
-- The CUDA compiler identification is NVIDIA 12.5.82 with host compiler GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- CUDA_ARCHITECTURES: native
-- Found Python

Run test executable

In [4]:
! ./build/test/cu/mxint/dequantize/test_mxint8_dequantize1d

Usage: ./build/test/cu/mxint/dequantize/test_mxint8_dequantize1d [m] [group_size] [is_random]
m=4096, group_size=128, num_groups=32, is_random=0
PASSED


## Build & Try mase-cuda Package

The building process can be slow. NVIDIA T4's compuate capability is 7.5, and L4 is 8.9.

In [5]:
! TORCH_CUDA_ARCH_LIST="7.5" MAX_JOBS=$(nproc --all) python -m build --wheel

* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - numpy
  - setuptools
  - torch
* Getting build dependencies for wheel...
running egg_info
creating src/mase_cuda.egg-info
writing src/mase_cuda.egg-info/PKG-INFO
writing dependency_links to src/mase_cuda.egg-info/dependency_links.txt
writing requirements to src/mase_cuda.egg-info/requires.txt
writing top-level names to src/mase_cuda.egg-info/top_level.txt
writing manifest file 'src/mase_cuda.egg-info/SOURCES.txt'
reading manifest file 'src/mase_cuda.egg-info/SOURCES.txt'
writing manifest file 'src/mase_cuda.egg-info/SOURCES.txt'
* Building wheel...
running bdist_wheel
running build
running build_py
creating build/lib.linux-x86_64-cpython-311/mase_cuda
copying src/mase_cuda/utils.py -> build/lib.linux-x86_64-cpython-311/mase_cuda
copying src/mase_cuda/__init__.py -> build/lib.linux-x86_64-cpython-311/mase_cuda
copying src/mase_cuda/constants.py -> build/lib.linux-x86_64-cpython-311/mase_cuda


> 🩹 We create a new env and run experiments there to avoid errors like `cuda pytorch undefined symbol` raied by the Colab's built-in Python

When the wheel is built, install the mase-cuda wheel into **a new dev env**

- Open **Colab termimal** and run the following commands to create a dev env and install mase-cuda:

  ```bash
  cd mase-cuda
  tox -e dev # create dev env
  . .tox/dev/bin/activate # activate dev env
  which pip # ensure this is the pip in .tox/dev
  pip install dist/mase_cuda-0.0.1-cp311-cp311-linux_x86_64.whl # install mase-cuda
  ```
- Colab terminal: Running the following command to profile dequantization latency (CPU vs GPU). This is slow:

  ```bash
  pytest -v --log-cli-level INFO test/py/mxint8/dequantize/test_dequantize1d.py::test_ext_dequantize1d_latency
  ```

The output looks like this

```bash
============================================================== test session starts ===============================================================
platform linux -- Python 3.11.11, pytest-8.3.4, pluggy-1.5.0 -- /content/mase-cuda/.tox/dev/bin/python
cachedir: .pytest_cache
rootdir: /content/mase-cuda
configfile: tox.ini
collected 1 item                                                                                                                                 

test/py/mxint8/dequantize/test_dequantize1d.py::test_ext_dequantize1d_latency
----------------------------------------------------------------- live log call ------------------------------------------------------------------
INFO     test_dequantize1d:test_dequantize1d.py:203
+-----------+------------+------------------------+------------------------+---------------------+
|     m     | group_size |      latency_cpu       |      latency_gpu       |     GPU speedup     |
+-----------+------------+------------------------+------------------------+---------------------+
|   1024    |     8      | 1.8215179443359376e-05 | 5.830879891291261e-05  | 0.31239160783546144 |
|   1024    |     16     | 1.1014938354492188e-05 | 2.411839971318841e-05  | 0.45670270355744236 |
|   1024    |     32     | 1.0752677917480469e-05 | 2.3455999884754422e-05 | 0.45841908127179576 |
|   1024    |     64     | 1.0418891906738282e-05 | 2.366719990968704e-05  | 0.44022495041645404 |
|   1024    |    128     | 1.043081283569336e-05  | 2.370399972423911e-05  | 0.44004442106987857 |
|   1024    |    256     | 9.298324584960938e-06  | 2.200479982420802e-05  | 0.42255892619989316 |
|   1024    |    512     | 9.393692016601562e-06  | 2.2529599815607072e-05 | 0.41694890692617675 |
|  2097152  |     8      |  0.019188427925109865  | 3.4844799526035784e-05 |  550.682689701584   |
|  2097152  |     16     |  0.018679165840148927  | 3.453759923577308e-05  |  540.8356762910598  |
|  2097152  |     32     |  0.018654394149780273  | 3.327519977465272e-05  |  560.6095313059608  |
|  2097152  |     64     |  0.01862926483154297   | 3.344159927219152e-05  |  557.068598302181   |
...
| 234881024 |     32     |   2.2112363457679747   |  0.008763614416122436  | 252.32013194235924  |
| 234881024 |     64     |   2.3624018669128417   |  0.008794196844100953  | 268.63190678947717  |
| 234881024 |    128     |   2.402256155014038    |  0.008851744079589843  |  271.3878907268803  |
| 234881024 |    256     |   2.4408880949020384   |  0.008949855947494508  |  272.7293164517759  |
| 234881024 |    512     |   2.4862973570823668   |  0.004568324756622315  |  544.2470685732645  |
+-----------+------------+------------------------+------------------------+---------------------+
PASSED                                                                                                                                     [100%]

========================================================= 1 passed in 466.97s (0:07:46) ==========================================================
```

In [6]:
!cd mase-cuda
!tox -e dev # create dev env

/bin/bash: line 1: cd: mase-cuda: No such file or directory
dev: install_deps> python -I -m pip install black build 'colorlog>=6.8.2' ipython ml_dtypes ninja 'numpy>=1.26.4' 'pytest>=8.2.0' 'scipy>=1.12.0' tabulate 'torch>=2.3.0'
  dev: OK (143.02 seconds)
  congratulations :) (143.36 seconds)


In [7]:
!. .tox/dev/bin/activate && which pip && pip install dist/mase_cuda-0.0.1-cp311-cp311-linux_x86_64.whl && pip install transformers && pytest -v --log-cli-level INFO test/py/mxint8/dequantize/test_dequantize1d.py::test_ext_dequantize1d_latency

/content/mase-cuda/.tox/dev/bin/pip
Processing ./dist/mase_cuda-0.0.1-cp311-cp311-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.8 MB/s eta 0:00:00
======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.4, pluggy-1.5.0 -- /content/mase-cuda/.tox/dev/bin/python
cachedir: .pytest_cache
rootdir: /content/mase-cuda
configfile: tox.ini
collected 1 item                                                                                   

test/py/mxint8/dequantize/test_dequantize1d.py::test_ext_dequantize1d_latency 
------------------------------------------ live log call -------------------------------------------
INFO     test_dequantize1d:

### FP32 Deberta Demo

- Colab Terminal: Install transformers in the dev env: `pip install transformers`

- demo.py: Copy the following codes into a new file demo.py

```python
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "AnkitAI/deberta-xlarge-base-emotions-classifier"
# if you meet OOM error, try this smaller model, but the quantization effect may not be obvious later
# model_name = "AnkitAI/deberta-v3-small-base-emotions-classifier"
model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda().eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)
label2emotion = {idx: emotion for emotion, idx in model.config.label2id.items()}

# Example usage
@torch.no_grad()
def predict_emotion(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.cuda() for k, v in inputs.items()}
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = logits.argmax(dim=1)
    predictions = label2emotion[predictions.item()]
    top3_values, top3_indices = torch.topk(logits, 3)
    top3_values = top3_values.cpu().tolist()
    top3_indices = top3_indices.cpu().tolist()
    return predictions, (top3_values, top3_indices)


text = "I'm so happy with the results!"
emotion, top3 = predict_emotion(model, tokenizer, text)

print("Index to Emotion Mapping:", label2emotion)
print("Input text:", text)
print("Detected Emotion:", emotion)
print(f"top3 logits: {top3[0]}, top3 indices: {top3[1]}")
```

- Colab Terminal: run demo.py in the dev env

The output looks like this:

```bash
config.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 1.12k/1.12k [00:00<00:00, 7.73MB/s]
model.safetensors: 100%|█████████████████████████████████████████████████████████████████████████████████████| 3.04G/3.04G [01:12<00:00, 42.0MB/s]
tokenizer_config.json: 100%|█████████████████████████████████████████████████████████████████████████████████| 1.31k/1.31k [00:00<00:00, 12.6MB/s]
vocab.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 798k/798k [00:00<00:00, 5.79MB/s]
merges.txt: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 456k/456k [00:00<00:00, 3.35MB/s]
tokenizer.json: 100%|████████████████████████████████████████████████████████████████████████████████████████| 2.11M/2.11M [00:00<00:00, 10.3MB/s]
special_tokens_map.json: 100%|███████████████████████████████████████████████████████████████████████████████████| 969/969 [00:00<00:00, 8.67MB/s]
Index to Emotion Mapping: {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
Input text: I'm so happy with the results!
Detected Emotion: joy
top3 logits: [[7.345228672027588, -1.4850201606750488, -1.6403964757919312]], top3 indices: [[1, 4, 0]]
```

In [8]:
!. .tox/dev/bin/activate && python ../demo.py

config.json: 100% 1.12k/1.12k [00:00<00:00, 5.19MB/s]
model.safetensors: 100% 3.04G/3.04G [01:12<00:00, 42.1MB/s]
tokenizer_config.json: 100% 1.31k/1.31k [00:00<00:00, 9.57MB/s]
vocab.json: 100% 798k/798k [00:00<00:00, 3.73MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 2.12MB/s]
tokenizer.json: 100% 2.11M/2.11M [00:00<00:00, 9.29MB/s]
special_tokens_map.json: 100% 969/969 [00:00<00:00, 7.43MB/s]
Index to Emotion Mapping: {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
Input text: I'm so happy with the results!
Detected Emotion: joy
top3 logits: [[7.3452301025390625, -1.4850208759307861, -1.6403967142105103]], top3 indices: [[1, 4, 0]]


### MXINT8 Deberta
- demo-q.py: Copy the following codes into a new file demo-q.py. This file creates the quantized Deberta and compare GPU memory usage of MXINT8 model with FP32 model

```python
import torch
import time
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from mase_cuda.mxint8.linear import QLinearPacked

init_memory = torch.cuda.memory_allocated()  # in bytes
model_name = "AnkitAI/deberta-xlarge-base-emotions-classifier"
model = AutoModelForSequenceClassification.from_pretrained(model_name, torch_dtype=torch.float32).cuda().eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)
label2emotion = {idx: emotion for emotion, idx in model.config.label2id.items()}

mxint8_group_size = 32
assert model.config.hidden_size % mxint8_group_size == 0
assert model.config.intermediate_size % mxint8_group_size == 0

text = "I'm so happy with the results!"

def timed_gpu(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result1, result2,  = fn()
    end.record()
    torch.cuda.synchronize()
    return result1, result2, start.elapsed_time(end) / 1000

def time_model(fn, model, tokenizer,  n=500):
    times = []
    sentences = [
    "I love this product! It works perfectly and exceeded my expectations.",  # Positive
    "This service is awful; I’ll never use it again.",  # Negative
    "The package arrived today at 3:00 PM.",  # Neutral
    "The sunset today was absolutely breathtaking!",  # Positive
    "I’m so frustrated with the constant delays and poor communication.",  # Negative
    "Great, another meeting that could’ve been an email.",  # Sarcastic/Passive-Aggressive
    "I cant believe it, thats soo bad!",
    "Interesting choice of design... not sure how I feel about it.",  # Ambiguous
    "This is the worst experience I’ve ever had—terrible customer support!",  # Strongly Negative
    "I'm so happy with the results!"
]
    result = [0,0,0,0,0,0,0,0,0,0]
    for i in range(n):
        result1, result2, t = timed_gpu(lambda: fn(model, tokenizer, sentences[i%10]))
        times.append(t)
        result[i%10] = result1
    avg_time = sum(times) / len(times)
    print(result)
    return result1, result2, avg_time


@torch.no_grad()
def predict_emotion(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.cuda() for k, v in inputs.items()}
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = logits.argmax(dim=1)
    predictions = label2emotion[predictions.item()]
    top3_values, top3_indices = torch.topk(logits, 3)
    top3_values = top3_values.cpu().tolist()
    top3_indices = top3_indices.cpu().tolist()
    return predictions, (top3_values, top3_indices)

# check the GPU memory usage of FP32 model
torch.cuda.reset_peak_memory_stats()
emotion_fp32, top3_fp32, time_fp32 = time_model(predict_emotion, model, tokenizer)
peak_memory_fp32 = torch.cuda.max_memory_allocated() - init_memory  # in bytes


def set_layer_by_name(module: torch.nn.Module, name: str, new_layer: torch.nn.Module):
    levels = name.split(".")
    if len(levels) > 1:
        mod_ = module
        for l_idx in range(len(levels) - 1):
            if levels[l_idx].isdigit():
                mod_ = mod_[int(levels[l_idx])]
            else:
                mod_ = getattr(mod_, levels[l_idx])
        setattr(mod_, levels[-1], new_layer)
    else:
        setattr(module, name, new_layer)


for layer_name, layer in model.named_modules():
    if not isinstance(layer, torch.nn.Linear):
        continue
    if "classifier" in layer_name:
        continue
    layer.cuda()
    layer_q = QLinearPacked.build_from_linear(layer, group_size=mxint8_group_size)
    set_layer_by_name(model, layer_name, layer_q)
    del layer
    torch.cuda.empty_cache()

torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
emotion_mxint8, top3_mxint8, time_mxint8 = time_model(predict_emotion, model, tokenizer)
peak_memory_mxint8 = torch.cuda.max_memory_allocated() - init_memory  # in bytes

print(f"FP32 model peak memory: {peak_memory_fp32/1024**2:.4f} MB")
print(f"FP32 single inference time: {time_fp32:.4f} s")
print(f"PF32 prediction: {emotion_fp32}")
print(f"FP32 top3 logits: {top3_fp32[0]}, indices: {top3_fp32[1]}")

print(f"MXINT8 model peak memory: {peak_memory_mxint8/1024**2:.4f} MB")
print(f"MXINT8 single inference time: {time_mxint8:.4f} s")
print(f"MXINT8 prediction: {emotion_mxint8}")
print(f"MXINT8 top3 logits: {top3_mxint8[0]}, indices: {top3_mxint8[1]}")
```

- Colab Terminal: Run demo-q.py in the dev env

The output looks like this:

```bash
FP32 model peak memory: 2906.1997 MB
PF32 prediction: joy
FP32 top3 logits: [[7.345228672027588, -1.4850201606750488, -1.6403964757919312]], indices: [[1, 4, 0]]
MXINT8 model peak memory: 976.1616 MB
MXINT8 prediction: joy
MXINT8 top3 logits: [[7.350157737731934, -1.488325834274292, -1.649757981300354]], indices: [[1, 4, 0]]
```

In [18]:
!. .tox/dev/bin/activate && python ../demo-q.py

['joy', 'anger', 'joy', 'joy', 'anger', 'joy', 'sadness', 'fear', 'anger', 'joy']
['joy', 'anger', 'joy', 'joy', 'anger', 'joy', 'sadness', 'fear', 'anger', 'joy']
FP32 model peak memory: 2908.2578 MB
FP32 single inference time: 0.1017 s
PF32 prediction: joy
FP32 top3 logits: [[7.3452301025390625, -1.4850208759307861, -1.6403967142105103]], indices: [[1, 4, 0]]
MXINT8 model peak memory: 978.0288 MB
MXINT8 single inference time: 0.1342 s
MXINT8 prediction: joy
MXINT8 top3 logits: [[7.350157737731934, -1.4883261919021606, -1.6497581005096436]], indices: [[1, 4, 0]]
